# Polymer + particle mixture using pyPRISM

In [5]:
import sys as s
s.path.append('/home/beth/Desktop/Programs/patchy_polymer_with_nanocrystal/prism')
import pyPRISM
import matplotlib.pyplot as plt
from custom_prism_tools.DiscreteKoyamaPartitioned import DiscreteKoyamaPartitioned
from custom_prism_tools.DiscreteKoyama import DiscreteKoyama
from custom_prism_tools.KovalenkoHirata import KovalenkoHirata
from custom_prism_tools.ShiftedWeeksChandlerAnderson import ShiftedWeeksChandlerAnderson
from custom_prism_tools.ThermodynamicsClosedForm import ThermodynamicsClosedForm
import numpy as np
%matplotlib inline

## Tunable parameters

In [6]:
class Params:
    def __init__(self):
        #parameters
        self.length = None
        self.particle_eta = None
        self.N_polymer__N_particle = None
        self.particle_diameter = None
        self.polymer_site_diameter = None 
        self.persistence_length_scale = None
        self.epsilon = None
        
    def calcDependentParams(self):
        self.length_scale = self.polymer_site_diameter
        self.particle_density = 6.0*self.particle_eta/(np.pi*self.particle_diameter**3.0)
        self.polymer_density = self.N_polymer__N_particle*self.particle_density
        self.polymer_site_density = self.length*self.polymer_density
        self.persistence_length = self.persistence_length_scale*self.polymer_site_diameter
        self.particle_polymer_cross_diameter = (self.particle_diameter + self.polymer_site_diameter)/2.0
        self.bond_length = 0.97 * self.polymer_site_diameter #kremer-grest

params=Params()
params.length = 10
params.particle_eta = 0.04
params.N_polymer__N_particle = 7.0
params.particle_diameter = 5.0
params.polymer_site_diameter = 1.0
params.persistence_length_scale = 4.0/3.0
params.epsilon = 1.0

params.calcDependentParams()

In [7]:
def setSystem(sys):
    # The composition of the system is desribed via number densities
    sys.density['particle'] = params.particle_density
    sys.density['polymer_end']  = params.polymer_site_density*(2.0/float(params.length))
    sys.density['polymer_middle']  = params.polymer_site_density*(float(params.length-2)/float(params.length))

    # The diameter of each site is specified (in reduced units)
    sys.diameter['particle'] = params.particle_diameter
    sys.diameter['polymer_end'] = params.polymer_site_diameter
    sys.diameter['polymer_middle'] = params.polymer_site_diameter
    
    sys.omega['polymer_end','polymer_end'] = DiscreteKoyamaPartitioned(sigma=params.polymer_site_diameter, 
                                                                   l=params.bond_length, 
                                                                   length=params.length, 
                                                                   lp=params.persistence_length, 
                                                                   types=('end','end'))
    sys.omega['polymer_middle','polymer_middle'] = DiscreteKoyamaPartitioned(sigma=params.polymer_site_diameter, 
                                                                         l=params.bond_length, 
                                                                         length=params.length, 
                                                                         lp=params.persistence_length, 
                                                                         types=('middle','middle'))
    sys.omega['polymer_end','polymer_middle'] = DiscreteKoyamaPartitioned(sigma=params.polymer_site_diameter, 
                                                                      l=params.bond_length, 
                                                                      length=params.length, 
                                                                      lp=params.persistence_length, 
                                                                      types=('end','middle'))
    
    # The site-site interactions are specified via classes which are lazily
    # evaluated during the PRISM-object creation

    sys.potential['particle','particle'] = ShiftedWeeksChandlerAnderson(epsilon=params.epsilon, 
                                                                    length_scale=params.length_scale,
                                                                    sigma=params.particle_diameter)
    sys.potential['particle','polymer_end'] = ShiftedWeeksChandlerAnderson(epsilon=params.epsilon, 
                                                                       length_scale=params.length_scale,
                                                                       sigma=params.particle_polymer_cross_diameter)
    sys.potential['particle','polymer_middle'] = ShiftedWeeksChandlerAnderson(epsilon=params.epsilon, 
                                                                          length_scale=params.length_scale,
                                                                          sigma=params.particle_polymer_cross_diameter)
    sys.potential['polymer_end','polymer_end'] = ShiftedWeeksChandlerAnderson(epsilon=params.epsilon, 
                                                                          length_scale=params.length_scale,
                                                                          sigma=params.polymer_site_diameter)
    sys.potential['polymer_middle','polymer_middle'] = ShiftedWeeksChandlerAnderson(epsilon=params.epsilon, 
                                                                                length_scale=params.length_scale,
                                                                                sigma=params.polymer_site_diameter)
    sys.potential['polymer_end','polymer_middle'] = ShiftedWeeksChandlerAnderson(epsilon=params.epsilon, 
                                                                             length_scale=params.length_scale,
                                                                             sigma=params.polymer_site_diameter)
    return sys

In [ ]:
params.polymer_site_diameter = 1.0
gr_results = []
F_P = []
# The system holds all information needed to set up a PRISM problem. We
# instantiate the system by specifying the site types and thermal energy
# level (kT, coarse-grained temperature) of the system.
sys = pyPRISM.System(['particle','polymer_middle','polymer_end'], kT=1.0)

# We must discretize Real and Fourier space
sys.domain = pyPRISM.Domain(dr=0.01,length=4096)

# The molecular structure is described via intra-molecular correlation
# functions (i.e. omegas)
sys.omega['particle','particle'] = pyPRISM.omega.SingleSite()
sys.omega['particle','polymer_end'] = pyPRISM.omega.InterMolecular()
sys.omega['particle','polymer_middle'] = pyPRISM.omega.InterMolecular()

# Closure approximations are also specified via classes
sys.closure['particle','particle'] = KovalenkoHirata()
sys.closure['particle','polymer_end'] = KovalenkoHirata()
sys.closure['particle','polymer_middle'] = KovalenkoHirata()
sys.closure['polymer_end','polymer_end'] = KovalenkoHirata()
sys.closure['polymer_middle','polymer_middle'] = KovalenkoHirata()
sys.closure['polymer_end','polymer_middle'] = KovalenkoHirata()

gamma_list = []
for i in np.arange(15.0, 8.9, -3.0):
    gamma_list.append(1.0/i)
for i in np.arange(6.0, 1.9, -1.0):
    gamma_list.append(1.0/i)
for i in np.arange(1.0, 6.1, 1.0):
    gamma_list.append(i)
gamma_list.append(7.5)
for i in np.arange(9.0, 15.1, 3.0):
    gamma_list.append(i)

volfrac_list = []
volfrac_list.append(0.005)
volfrac_list.append(0.01)
volfrac_list.append(0.03)
#for i in np.arange(0.05, 0.51, 0.05):
#    volfrac_list.append(i)
    
guess=np.zeros(len(sys.types)*len(sys.types)*4096)
guess_save=guess

for params.length in range(3,5,1):
    guess=np.zeros(len(sys.types)*len(sys.types)*4096)
    for params.persistence_length_scale in [1.334, 1.43, 2, 2.5, 3, 4, 5, 10]:
        guess=np.zeros(len(sys.types)*len(sys.types)*4096)
        for params.particle_diameter in np.arange(8.0, 2.5, -1.0):
            guess=np.zeros(len(sys.types)*len(sys.types)*4096)
            filename="gr_"+str(params.length)+"_"+str(params.persistence_length_scale)+"_"+str(params.particle_diameter)+".dat"
            for params.particle_eta in volfrac_list: #np.arange(0.01, 0.11, 0.02): 
                if (params.particle_eta - volfrac_list[0]) > 0.001:
                    guess=guess_save
                for params.N_polymer__N_particle in gamma_list:
                    #[1./15., 1./12., 1./9., 1./6., 1./5., 0.25, 1./3., 0.5, 1., 2.]: 
#            if abs(params.particle_eta - 0.09) < 0.001 or params.N_polymer__N_particle == 0.1: 
                    eta_total = params.particle_eta*(1.+(params.polymer_site_diameter/params.particle_diameter)**3.*params.N_polymer__N_particle*params.length)
                    if eta_total < 0.5:
                        filename="PRISM_"+str(params.length)+"_"+str(params.persistence_length_scale)+"_"+str(params.particle_diameter)+"_"+str(params.particle_eta)+"_"+str(params.N_polymer__N_particle)+".dat"
                        print('==> Solving for variable=', params.length, params.persistence_length_scale, params.particle_diameter, params.particle_eta, params.N_polymer__N_particle)
                        params.calcDependentParams()
                        sys=setSystem(sys)
# Calling the .solve() method of the system object attempts to numerically
# solv the PRISM equation and, if successful, it returns a PRISM object
# containing all of the solved correlation functions.
                        PRISM = sys.solve(guess=guess, method='krylov', tol=1e-7, options={'disp':True, 'maxiter':50})
                        guess = np.copy(PRISM.x)

                        x = sys.domain.r[4:700:5]
                        y = pyPRISM.calculate.pair_correlation(PRISM)['particle','polymer_end'][4:700:5]
                        np.savetxt("/home/beth/Desktop/Programs/patchy_polymer_with_nanocrystal/prism/work/PRISMdata/"+filename, zip(x,y))
                        currentFP=ThermodynamicsClosedForm(PRISM, default_method='KH')
                        F_P.append(currentFP)
                        with open("/home/beth/Desktop/Programs/patchy_polymer_with_nanocrystal/prism/work/PRISMdata/"+filename, "a") as fh:
                            fh.write(str(currentFP)+"\n")
#                        gr_results.append([params.particle_eta,params.length,params.persistence_length_scale,params.N_polymer__N_particle,params.particle_diameter,x,y,currentFP])
                        if (params.N_polymer__N_particle - gamma_list[0]) < 0.001:
                            guess_save = np.copy(PRISM.x)
    print('')
    
print('Done!')
F_P = np.array(F_P)


('==> Solving for variable=', 3, 1.334, 8.0, 0.005, 0.066666666666666666)
0:  |F(x)| = 8.03415; step 1; tol 0.00156842
1:  |F(x)| = 0.33518; step 1; tol 0.00156646
2:  |F(x)| = 0.00150695; step 1; tol 1.8192e-05
3:  |F(x)| = 6.351e-08; step 1; tol 1.59857e-09
4:  |F(x)| = 7.32988e-14; step 1; tol 1.19882e-12
('==> Solving for variable=', 3, 1.334, 8.0, 0.005, 0.083333333333333329)
0:  |F(x)| = 1.24126e-08; step 1; tol 6.40645e-12
1:  |F(x)| = 4.23366e-14; step 1; tol 1.04701e-11
('==> Solving for variable=', 3, 1.334, 8.0, 0.005, 0.1111111111111111)
0:  |F(x)| = 2.27302e-08; step 1; tol 7.734e-12
1:  |F(x)| = 4.57228e-14; step 1; tol 3.64167e-12
('==> Solving for variable=', 3, 1.334, 8.0, 0.005, 0.16666666666666666)
0:  |F(x)| = 1.60551e-07; step 1; tol 9.64643e-11
1:  |F(x)| = 6.67398e-13; step 1; tol 1.5552e-11
('==> Solving for variable=', 3, 1.334, 8.0, 0.005, 0.20000000000000001)
0:  |F(x)| = 2.90458e-08; step 1; tol 8.77006e-12
1:  |F(x)| = 4.21392e-14; step 1; tol 1.8943e-12
('

In [ ]:
guess = np.copy(PRISM.x)

In [ ]:
rdf = pyPRISM.calculate.pair_correlation(PRISM)
# Plot the results using matplotlib
plt.plot(sys.domain.r,rdf['particle','particle'],color='gold',lw=1.25,ls='-')
plt.plot(sys.domain.r,rdf['particle','polymer_end'],color='red',lw=3,ls='-')
plt.plot(sys.domain.r,rdf['particle','polymer_middle'],color='blue',lw=3,ls='-')
plt.plot(sys.domain.r,rdf['polymer_end','polymer_end'],color='green',lw=1.25,ls='-')
plt.plot(sys.domain.r,rdf['polymer_middle','polymer_middle'],color='purple',lw=1.25,ls='-')
plt.plot(sys.domain.r,rdf['polymer_end','polymer_middle'],color='orange',lw=1.25,ls='-')
plt.ylabel('pair correlation')
plt.xlabel('separation distance')
plt.xlim(0,15)
plt.ylim(0,3)
plt.show()

In [ ]:
color_idx = np.linspace(0, 1, len(gr_results))
color_idx = np.linspace(0, 1, len(gr_results))
#for _,r,gr in gr_results:
for i, gr_obj in zip(color_idx, gr_results):
    eta_c,l,l_p,gamma,d_c,r,gr,_=gr_obj
    if abs(eta_c - 0.09) < 0.001 and abs(d_c - 5.0) < 0.001 and abs(gamma - 4.0) < 100.0:
        plt.plot(r,gr,lw=1.25, color=plt.cm.cool(i),ls='-')
plt.ylabel('pair correlation')
plt.xlabel('separation distance')
plt.xlim(2.5,8)
#plt.ylim(0,2)
plt.show()

In [ ]:
print F_P

In [ ]:
print gr_results

## k-space contribution to free energy

In [ ]:
Ck = np.ones((len(sys.domain.k), 1+length,1+length), dtype=None, order='C')
Wk = np.ones((len(sys.domain.k), 1+length,1+length), dtype=None, order='C')
p = np.zeros((1+length,1+length), dtype=None, order='C')
k = sys.domain.k

types = ['particle', 'polymer_end', 'polymer_middle']
type_indices = {'particle': [1], 'polymer_end': [1,length], 'polymer_middle': range(2, length)}
single_site_density = {'particle': particle_density, 
                       'polymer_end': polymer_density, 
                       'polymer_middle': polymer_density}

#loops over the rows
row = 0
for type_1 in types:
    for index_1 in type_indices[type_1]:
        
        #loops over the columns
        column = 0
        for type_2 in types:
            for index_2 in type_indices[type_2]:
                
                #build a 3d entry into our tensor
                c = PRISM.directCorr[type_1,type_2]
                Ck[:,row:row+1,column:column+1] = np.expand_dims(np.expand_dims(c, axis=1), axis=1)
                
                #polymer-polymer w(k)
                if type_1 in ['polymer_end', 'polymer_middle'] and type_2 in ['polymer_end', 'polymer_middle']:
                    n = abs(index_1-index_2)
                    if n:
                        w = sys.omega[type_1,type_2].koyama_kernel_fourier(k,n)
                    else:
                        w = np.ones((len(sys.domain.k)), dtype=None, order='C')
                else:
                    w = sys.omega[type_1,type_2].calculate(k)
                    
                Wk[:,row:row+1,column:column+1] = np.expand_dims(np.expand_dims(w, axis=1), axis=1)
                
                p[row:row+1,column:column+1] = (row == column)*single_site_density[type_1]
            
                column += 1
            
        row+=1

In [ ]:
trace_1 = np.trace(np.matmul(np.matmul(Wk, Ck), p), axis1=1, axis2=2)
trace_2 = np.trace(np.matmul(np.matmul(p, Wk), Ck), axis1=1, axis2=2)

In [ ]:
I=np.repeat(np.expand_dims(np.identity(1+length), axis=0), repeats=len(sys.domain.k), axis=0)
lndet_1 = np.log(np.linalg.det(I-np.matmul(np.matmul(Wk, Ck), p)))
lndet_2 = np.log(np.linalg.det(I-np.matmul(np.matmul(p, Wk), Ck)))

In [ ]:
F = 4.0*np.pi/(2.0*(2.0*np.pi)**3)*np.trapz(k*k*(trace_1+lndet_1), k)
print F

#### Testing the PRISM version

In [ ]:
sys.density.types

In [ ]:
num_types = len(sys.density.types)

Ck = np.ones((len(sys.domain.k), num_types, num_types), dtype=None, order='C')
Wk = np.ones((len(sys.domain.k), num_types, num_types), dtype=None, order='C')
p = np.zeros((num_types,num_types), dtype=None, order='C')
k = sys.domain.k

#loops over the rows
row = 0
for type_1 in sys.density.types:
        
    #loops over the columns
    column = 0
    for type_2 in sys.density.types:
                
        #build a 3d entry into our tensor
        c = PRISM.directCorr[type_1,type_2]
        Ck[:,row:row+1,column:column+1] = np.expand_dims(np.expand_dims(c, axis=1), axis=1)
        
        w = PRISM.omega[type_1,type_2]
        Wk[:,row:row+1,column:column+1] = np.expand_dims(np.expand_dims(w, axis=1), axis=1)
                
        p[row:row+1,column:column+1] = ((row == column)*sys.density[type_1] + 
                                        (row != column)*(sys.density[type_1] + sys.density[type_2]))
            
        column += 1
            
    row+=1

In [ ]:
trace = np.trace(np.matmul(Wk, Ck), axis1=1, axis2=2)

In [ ]:
I=np.repeat(np.expand_dims(np.identity(num_types), axis=0), repeats=len(sys.domain.k), axis=0)
lndet = np.log(np.linalg.det(I-np.matmul(Wk, Ck)))

## k-space free energy

In [ ]:
import pyPRISM
import numpy as np
from itertools import combinations_with_replacement
from copy import deepcopy
import warnings

def ExcessFreeEnergy(PRISM, default_method='HNC'): 
    
    #check on the closures and raise a warning of incompatible
    closures = set()
    for type_1, type_2 in combinations_with_replacement(PRISM.sys.types, 2):
        closures.add(type(PRISM.sys.closure[type_1, type_2]))
    
    if not (closures - set([type(KovalenkoHirata())])):
        method = 'KH'
    elif not (closures - set([type(pyPRISM.closure.HyperNettedChain())])):
        method = 'HNC'
    else:
        warnings.warn('This calculation is only theoretically valid if either \\
                      the KovalenkoHirata (KH) or HyperNettedChain (HNC) \\
                      closures are used for all pairs. Defaulting to {} \\
                      functional form for use with mixed closures.'.format(default_method), UserWarning)
        method = default_method
    
    #extra quantities for k-space calculation
    k = PRISM.sys.domain.k
    I = pyPRISM.IdentityMatrixArray(length=len(PRISM.sys.domain.k), rank=3, 
                                    types=PRISM.sys.types, space=pyPRISM.Space.Fourier)
    
    #extra quantities for r-space calculation
    r = PRISM.sys.domain.r
    p = np.diag(np.diag(PRISM.sys.density.site.data[0]))  #this is a diagonal density matrix
    
    #direct correlation function in r- and k-space
    if PRISM.directCorr.space == pyPRISM.Space.Real:
        Cr = deepcopy(PRISM.directCorr)
        Ck = deepcopy(Cr)
        PRISM.sys.domain.MatrixArray_to_fourier(Ck)
    elif PRISM.directCorr.space == pyPRISM.Space.Fourier:
        Ck = deepcopy(PRISM.directCorr)
        Cr = deepcopy(Ck)
        PRISM.sys.domain.MatrixArray_to_real(Cr)
    
    #intramolecular structure in k-space
    Wk = deepcopy(PRISM.omega)
    if Wk.space == pyPRISM.Space.Real:
        PRISM.sys.domain.MatrixArray_to_fourier(Wk)
    
    #total correlation function in r-space
    Hr = deepcopy(PRISM.totalCorr)
    if Hr.space == pyPRISM.Space.Fourier:
        PRISM.sys.domain.MatrixArray_to_real(Hr)
    
    #calculate the k-space contribution
    tr_WkCk = np.trace(Wk.dot(Ck).data, axis1=1, axis2=2)
    lndet_WkCk = np.log( np.linalg.det( (I-Wk.dot(Ck)).data ) )
    F_ex_k = 4.0*np.pi/(2.0*(2.0*np.pi)**3)*np.trapz(k*k*(tr_WkCk + lndet_WkCk), k)
    
    #calculate the r-space contribution
    if method == 'KH':
        structure = ((1.0/2.0)*(Hr*Hr).data*np.heaviside(-Hr.data, 0) - Cr.data)
    elif method == 'HNC':
        structure = ((1.0/2.0)*(Hr*Hr).data - Cr.data)
    kernel_r = np.sum(np.matmul(np.matmul(p, structure), p), axis=(1,2))
    F_ex_r = (4.0*np.pi/2.0)*np.trapz(r*r*kernel_r, r)
    
    #total free energy per unit volume and kBT
    F_ex = F_ex_r + F_ex_k
    
    return F_ex

In [ ]:
ExcessFreeEnergy(PRISM)/PRISM.sys.density.total

In [ ]:
ExcessFreeEnergy(PRISM)/PRISM.sys.density.total

In [ ]:
type(PRISM.sys.closure['particle','polymer_middle']) == type(KovalenkoHirata())

In [ ]:
PRISM.sys.types

In [ ]:
a = set()

In [ ]:
a.add(3)

In [ ]:
not (a - set([3,4]))

In [ ]:
k = sys.domain.k
I = pyPRISM.IdentityMatrixArray(length=len(sys.domain.k), rank=3, types=sys.types, space=pyPRISM.Space.Fourier)
Ck = PRISM.directCorr
Wk = PRISM.omega
tr_WkCk = np.trace(Wk.dot(Ck).data, axis1=1, axis2=2)
lndet_WkCk = np.log( np.linalg.det( (I-Wk.dot(Ck)).data ) )
F_ex_k = -4.0*np.pi/(2.0*(2.0*np.pi)**3)*np.trapz(k*k*(tr_WkCk + lndet_WkCk), k)

In [ ]:
from copy import deepcopy

In [ ]:
r = sys.domain.r
Cr = deepcopy(Ck)
sys.domain.MatrixArray_to_real(Cr)
p = np.diag(np.diag(sys.density.site.data[0]))
Hr = PRISM.totalCorr
structure = ((1.0/2.0)*(Hr*Hr).data*np.heaviside(-Hr.data, 0) - Cr.data)
kernel_r = np.sum(np.matmul(np.matmul(p, structure), p), axis=(1,2))
F_ex_r = (4.0*np.pi/2.0)*np.trapz(r*r*kernel_r, r)

In [ ]:
print F_ex_r 
print F_ex_k
print F_ex_r - F_ex_k

In [ ]:
def CS(n):
    return ((4.0*n-3.0*n*n)/(1.0-n)**2)

In [ ]:
CS(0.4)

In [ ]:
#trace_1 = np.trace(np.matmul(np.matmul(Wk, Ck), p), axis1=1, axis2=2)
trace = np.trace(np.matmul(Wk, Ck), axis1=1, axis2=2)

In [ ]:
I=np.repeat(np.expand_dims(np.identity(num_types), axis=0), repeats=len(sys.domain.k), axis=0)
lndet = np.log(np.linalg.det(I-np.matmul(Wk, Ck)))

In [ ]:
plt.plot(trace)
plt.plot(lndet)
plt.ylim(-10,10)

In [ ]:
F = 4.0*np.pi/(2.0*(2.0*np.pi)**3)*np.trapz(k*k*(trace+lndet), k)
print F

In [ ]:
plt.plot(abs(trace_1))
plt.plot(abs(trace))
plt.yscale('log')

In [ ]:
plt.plot(PRISM.omega['polymer_end','polymer_end'])

In [ ]:
PRISM.omega['particle','particle']

## r-space contribution to free energy

In [ ]:
from copy import deepcopy

In [ ]:
directCorr = deepcopy(PRISM.directCorr)
totalCorr = deepcopy(PRISM.totalCorr)
sys.domain.MatrixArray_to_real(directCorr)

In [ ]:
types = ['particle', 'polymer_end', 'polymer_middle']
num_types = len(types)

Cr = np.ones((len(sys.domain.k), num_types, num_types), dtype=None, order='C')
Hr = np.ones((len(sys.domain.k), num_types, num_types), dtype=None, order='C')
r = sys.domain.r

for type_1 in types:
    for type_2 in types:
        

In [ ]:
plt.plot(PRISM.totalCorr['particle','polymer_end'])

In [ ]:
plt.scatter(sys.domain.r, C['particle','particle'])
plt.xlim(0,4)

In [ ]:
lndet_1

In [ ]:
plt.plot(lndet_1)
plt.plot(trace_1)
plt.plot(lndet_1+trace_1)
#plt.yscale('log')
#plt.ylim(-1,1)

In [ ]:
print p

In [ ]:
plt.plot(trace)
#plt.plot(lndet)

In [ ]:
I.shape

In [ ]:
I=np.repeat(np.expand_dims(np.identity(1+length), axis=0), repeats=len(sys.domain.k), axis=0)

In [ ]:
dkp = DiscreteKoyamaPartitioned(sigma=0.01, 
                                l=polymer_site_diameter, 
                                length=length, 
                                lp=0.20013,
                                types=('end','middle'))

In [ ]:
print dkp.epsilon
print dkp.cos1
print dkp.cos2

In [ ]:
print dkp.epsilon
print dkp.cos1
print dkp.cos2

In [ ]:
4*(0.2)**3/(4*0.2**2-0.01**2)

In [ ]:
polymer_site_diameter*3.0005/3.0

In [ ]:
sigma = 0.2
l=0.2

1 - sigma*sigma/(2.0 * l * l)

In [ ]:
1/6.

In [ ]:
def Cos1(epsilon, cos0):
    e = epsilon
    return (1/e  - ( np.exp(e) + cos0*np.exp(-e*cos0) )/( np.exp(e) - np.exp(-e*cos0) ))

In [ ]:
l = 0.5
lp = (2.9/3.0)*l
sigma = 0.001
cos0 = 1.0 - sigma*sigma/(2.0*l*l)
cos1 = l/lp - 1.0

In [ ]:
from scipy.optimize import root

In [ ]:
func = lambda e: Cos1(e, cos0) - cos1

In [ ]:
sol = root(func, [0.1], jac=False, method='hybr')
sol.x

In [ ]:
w_bad = pyPRISM.omega.DiscreteKoyama(sigma=1.0, 
                                        l=1.0, 
                                        length=50, 
                                        lp=4./3.0 )
w_good = DiscreteKoyama(sigma=1.0, 
                        l=0.5001, 
                        length=50, 
                        lp=50)

print (w_good.cos1,w_good.cos2,w_good.epsilon)

In [ ]:
w_good = DiscreteKoyama(sigma=1.0, 
                        l=0.61, 
                        length=50, 
                        lp=1.86083)

print (w_good.cos1,w_good.cos2,w_good.epsilon)

In [ ]:
w_good = DiscreteKoyama(sigma=1.0, 
                        l=0.61, 
                        length=50, 
                        lp=1.8608353)

print (w_good.cos1,w_good.cos2,w_good.epsilon)

In [ ]:
k = PRISM.sys.domain.k
plt.plot(k, k*k*w_bad.calculate(k))
plt.plot(k, k*k*w_good.calculate(k), color='red')
plt.xlim(0,4)
plt.ylim(0,10)